In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import json
import polars as pl
from polars import col, lit
import altair as alt

In [2]:
with open("../../output/last_run_tabular.json") as f:
    data = json.load(f)
agents = pl.DataFrame(data["agents"])
markets = pl.DataFrame(data["markets"]).select(pl.exclude("pricer"))
actions = pl.DataFrame(data["actions"])

In [3]:
actions

action,agent_id,good,port_id,tick
str,str,str,str,i64
"""BuyAndMove""","""A""","""Wheat""","""Rome""",0
"""Noop""","""B""",null,null,0
"""Sell""","""A""","""Wheat""",null,1
"""Noop""","""B""",null,null,1
"""BuyAndMove""","""A""","""Wheat""","""Genoa""",2
"""Move""","""B""",null,"""Rome""",2


In [4]:
markets.head(3)

consumption,good,port,price,production,supply,tick
f64,str,str,f64,f64,f64,i64
30.0,"""Wheat""","""Rome""",80.0,20.0,100.0,0
20.0,"""Wheat""","""Genoa""",80.0,25.0,100.0,0
20.0,"""Wheat""","""Genoa""",79.0,25.0,105.0,1


In [5]:
markets.groupby(col("tick")).agg((pl.median("price").alias("median"), pl.sum("price").alias("sum")))

tick,median,sum
i64,f64,f64
2,81.0,162.0
3,81.5,163.0
1,80.5,161.0
0,80.0,160.0


In [6]:
alt.Chart(markets).mark_point().encode(x="tick:Q", y="price:Q")


alt.Chart(...)

In [7]:
alt.Chart(agents).mark_point().encode(x="tick:Q", y="coins:Q")

alt.Chart(...)

In [8]:
agents.join(actions, left_on="id", right_on="agent_id").select(pl.exclude("id")).head(3)

behavior,cargo,coins,pos,tick,action,good,port_id,tick_right
str,str,f64,str,i64,str,str,str,i64
"""Greedy""",null,100.0,"""Genoa""",0,"""BuyAndMove""","""Wheat""","""Rome""",0
"""Greedy""",null,100.0,"""Genoa""",0,"""Sell""","""Wheat""",null,1
"""Greedy""",null,100.0,"""Genoa""",0,"""BuyAndMove""","""Wheat""","""Genoa""",2
